In [1]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.get(name="quick-starts-ws-121548")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121548
Azure region: southcentralus
Subscription id: 422c78cb-7b9f-41e7-a94c-e144f7afde5e
Resource group: aml-quickstarts-121548


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
#from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = "aml-compute"
vm_size = "Standard_D2_V2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size, 
                                                                min_nodes = 0,
                                                                max_nodes = 4)
    # create the compute target
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

Found compute target: aml-compute


In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.95, 1.05),
        '--max_iter': uniform(95, 105)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

#import shutil
#project_folder = './training'
#shutil.copy('train.py', project_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './',
              entry_script = 'train.py',
              compute_target = compute_target,
              pip_packages = ['azureml-dataset-runtime[pandas,fuse]','azureml-defaults'],
              conda_packages = ['scikit-learn']
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est, 
                             hyperparameter_sampling = ps,
                             policy = policy,
                             primary_metric_name='Accuracy', 
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs = 20,
                             max_concurrent_runs = 4
)

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['scikit-learn']. We cannot guarantee image build will succeed.


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_4234de4f-5c9d-4162-ad15-e30c5181017c',
 'target': 'aml-compute',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-19T17:06:41.14184Z',
 'endTimeUtc': '2020-10-19T17:14:14.827985Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/422c78cb-7b9f-41e7-a94c-e144f7afde5e/resourceGroups/aml-quickstarts-121548/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-121548/experiments/**SCRUBBED**/runs/HD_4234de4f-5c9d-4162-ad15-e30c5181017c_0, {\'error\': {\n    "additional_properties": {\n        "componentName": "execution-worker"\n    },\n    "error": {\n        "additional_properties": {\n            "messageFormat": null,\n            "referenceCode": null,\n            "messageParameters": null,\n            "severity": null\n        },\n        "code": "UserError",\n        "message

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n Regularization Strength:',parameter_values[3])
print('\n Max iterations:',parameter_values[5])
print(best_run.get_file_names())

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset = TabularDatasetFactory.from_delimited_files(path = web_path)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)


In [ ]:
df = ds.to_pandas_dataframe()
df.head()

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'Accuracy',
    training_data = x_train,
    label_column_name = 'y',
    n_cross_validations = 5)

In [ ]:
# Submit your automl run

from azureml.core.experiment import Experiment

experiment = Experiment(ws, "automl_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)